# Weboldal Tartalom Feldolgozása és Kategorizálása

## 0. Könyvtárak Importálása és Szükséges Csomagok Telepítése

In [1]:
# Szükséges csomagok telepítése (Colab/Jupyter környezetben)
!pip install requests beautifulsoup4 pandas lxml unicodedata2
!pip install huspacy
!pip install transformers sentencepiece torch
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [2]:
import requests                     # HTTP kérésekhez (weboldal letöltése)
from bs4 import BeautifulSoup        # HTML parse-oláshoz
import pandas as pd                 # Adatkezeléshez (DataFrame)
import re                           # Reguláris kifejezésekhez
import unicodedata                  # Unicode normalizáláshoz
from functools import lru_cache

import huspacy
#huspacy.download() # Ha lokálisan futtatod és még nincs letöltve a modell, vagy ha a lentebbi try-except nem működik

import spacy                        # NLP feladatokhoz (lemmatizálás)
from sentence_transformers import SentenceTransformer # Beágyazások generálásához
import torch                        # PyTorch (gyakran a transformers könyvtárak függősége)
from transformers import pipeline   # Szövegkategorizáláshoz

# Figyelmeztetések kikapcsolása (opcionális)
import warnings
warnings.filterwarnings('ignore')

['/usr/bin/python3', '-m', 'pip', 'install', 'hu_core_news_lg @ https://huggingface.co/huspacy/hu_core_news_lg/resolve/v3.8.0/hu_core_news_lg-any-py3-none-any.whl']


### Globális Beállítások és Modellek Betöltése

In [11]:
# Magyar spaCy modell betöltése huspacy segítségével
nlp_hu = None
try:
    nlp_hu = huspacy.load()
    print("Magyar spaCy modell (huspacy alapú) sikeresen betöltve.")
except OSError:
    print("HIBA: A magyar spaCy modell (pl. hu_core_news_lg) nincs telepítve vagy nem tölthető be.")
    print("Próbáld meg futtatni a 'huspacy.download()' parancsot egy külön cellában, majd indítsd újra a futási környezetet.")
    print("Google Colab-ban futtasd: !python -m spacy download hu_core_news_lg, majd Runtime -> Restart session.")

# A cél URL
TARGET_URL = "https://www.aiwc.ca/blog/duckweed-a-superfood-from-the-wetlands/"

# Gyorsítótárazott lemmatizálás
@lru_cache(maxsize=1024)
def lemmatize_text_cached(text):
    # Ezt a függvényt a kód nem használja közvetlenül, de jó mintaként szolgálhat
    # A tényleges lemmatizálás a 'lemmatize_text_with_spacy' vagy 'lemmatize_texts_batch' függvényekben történik.
    if nlp_hu:
        doc = nlp_hu(str(text))
        lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
        return " ".join(lemmas)
    return str(text) # Fallback, ha nincs nlp_hu

Magyar spaCy modell (huspacy alapú) sikeresen betöltve.


## 1. HTML Tartalom Letöltése

In [12]:
print(f"\n--- 1. HTML Tartalom Letöltése: {TARGET_URL} ---")
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
html_content_article = None
try:
    response = requests.get(TARGET_URL, headers=headers, timeout=15)
    response.raise_for_status()
    html_content_article = response.text
    print(f"Az oldal sikeresen letöltve (hossz: {len(html_content_article)} karakter).")
except requests.exceptions.RequestException as e:
    print(f"Hiba történt az oldal letöltése közben: {e}")


--- 1. HTML Tartalom Letöltése: https://www.aiwc.ca/blog/duckweed-a-superfood-from-the-wetlands/ ---
Az oldal sikeresen letöltve (hossz: 295471 karakter).


## 2. Fő Cikkszöveg Kinyerése a HTML-ből

In [13]:
print("\n--- 2. Fő Cikkszöveg Kinyerése ---")
extracted_article_text_parts = []
if html_content_article:
    soup = BeautifulSoup(html_content_article, "lxml")

    article_body_specific = soup.find('div', class_='post-content')
    if article_body_specific:
        print("Specifikus 'post-content' div alapján történő kinyerés.")
        paragraphs = article_body_specific.find_all('p')
    elif soup.find('article'):
        print("Általános '<article>' tag alapján történő kinyerés.")
        article_body = soup.find('article')
        paragraphs = article_body.find_all('p')
    elif soup.find('div', role='article'):
        print("Általános 'div[role=\"article\"]' tag alapján történő kinyerés.")
        article_body = soup.find('div', role='article')
        paragraphs = article_body.find_all('p')
    else:
        print("Figyelem: Nem találtuk a szokásos cikk konténereket. Próbálkozás az összes <p> taggel a body-ból.")
        paragraphs = soup.find_all('p') if soup.body else []

    if paragraphs:
        print(f"Talált cikktartalom {len(paragraphs)} bekezdéssel.")
        relevant_paragraphs = [p.get_text(separator=" ", strip=True) for p in paragraphs]
        extracted_article_text_parts.extend(relevant_paragraphs)
    else:
        print("Nem sikerült bekezdéseket (<p>) kinyerni a fő tartalomból.")

    full_extracted_text = " ".join(filter(None, extracted_article_text_parts))

    if not full_extracted_text.strip() and html_content_article:
        print("Figyelem: A célzott kinyerés nem adott eredményt, próbálkozás a teljes body szövegével.")
        full_extracted_text = soup.body.get_text(separator=" ", strip=True) if soup.body else "Nem sikerült szöveget kinyerni."
else:
    full_extracted_text = "Az oldal HTML tartalma nem érhető el."

print(f"Kinyert nyers szöveg hossza: {len(full_extracted_text)} karakter.")
print("Kinyert szöveg eleje (max 500 karakter):")
print(full_extracted_text[:500] + "..." if len(full_extracted_text) > 500 else full_extracted_text)


--- 2. Fő Cikkszöveg Kinyerése ---
Általános '<article>' tag alapján történő kinyerés.
Talált cikktartalom 1 bekezdéssel.
Kinyert nyers szöveg hossza: 136 karakter.
Kinyert szöveg eleje (max 500 karakter):
By Natalie Galan In the heart of Alberta, the bison, both plains and wood bison, are more than just iconic symbols of the Canadian West.


## 3. Szöveg Tisztítása és Előfeldolgozása

In [14]:
print("\n--- 3. Szöveg Tisztítása és Előfeldolgozása ---")

# Adat DataFrame-be helyezése
df_article_processed = pd.DataFrame({'id': [TARGET_URL], 'raw_extracted_text': [full_extracted_text]})

# --- Segédfüggvények a tisztításhoz és feldolgozáshoz ---
def clean_special_chars_and_normalize(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = unicodedata.normalize("NFKC", text) # Unicode normalizálás
    text = re.sub(r"\[\d+\]", "", text) # [1], [23] stílusú hivatkozások eltávolítása
    text = re.sub(r"\[\d+-\d+\]", "", text) # [1-5] stílusú hivatkozások eltávolítása
    text = re.sub(r"http\S+", "", text) # URL-ek eltávolítása
    text = re.sub(r"[^a-zA-Z0-9áéíóöőúüűÁÉÍÓÖŐÚÜŰ\s.,!?:;\-]", " ", text)
    text = " ".join(text.split()) # Extra szóközök eltávolítása
    return text.strip()

def lemmatize_texts_batch(texts, nlp_model):
    if not nlp_model:
        return texts # Visszaadjuk az eredeti szövegeket, ha nincs modell
    # A huspacy.load() által visszaadott objektum maga a pipeline
    docs = nlp_model.pipe(texts, batch_size=16, disable=["parser", "ner"]) # Gyorsabb NER és parser nélkül
    lemmatized_texts = []
    for doc in docs:
        lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
        lemmatized_texts.append(" ".join(lemmas))
    return lemmatized_texts

def split_text_to_chunks(text, max_chunk_tokens=500, tokenizer=None):
    if not tokenizer:
        raise ValueError("A 'tokenizer' paraméter megadása kötelező a split_text_to_chunks függvényhez.")

    sentences = re.split(r'(?<=[.!?])\s+', text) # Mondatbontás
    chunks = []
    current_chunk_sentences = []

    for sentence in sentences:
        if not sentence.strip():
            continue
        temp_chunk_text = " ".join(current_chunk_sentences + [sentence])
        n_tokens = len(tokenizer.encode(temp_chunk_text))

        if n_tokens <= max_chunk_tokens:
            current_chunk_sentences.append(sentence)
        else:
            if current_chunk_sentences:
                chunks.append(" ".join(current_chunk_sentences).strip())

            if len(tokenizer.encode(sentence)) <= max_chunk_tokens:
                current_chunk_sentences = [sentence]
            else: # Ha egyetlen mondat is túl hosszú
                # print(f"Figyelem: Egyetlen mondat ({len(tokenizer.encode(sentence))} token) hosszabb, mint max_chunk_tokens ({max_chunk_tokens}). Durva vágás lehet szükséges.")
                # A SentenceTransformer encode levágja, ha a mondat maga hosszabb a modell max_seq_length-jénél
                chunks.append(sentence.strip())
                current_chunk_sentences = []

    if current_chunk_sentences:
        chunks.append(" ".join(current_chunk_sentences).strip())

    chunks = [ch for ch in chunks if ch]
    return chunks


def get_average_embedding(text, model, max_chunk_tokens=500): # model itt az sbert_model
    if not text or not text.strip():
        print("Üres szöveg a beágyazáshoz.")
        try:
            dim = model.get_sentence_embedding_dimension()
            return torch.zeros(dim).numpy()
        except: return None

    chunks = split_text_to_chunks(text, max_chunk_tokens=max_chunk_tokens, tokenizer=model.tokenizer)
    if not chunks:
        print("Figyelem: Nem sikerült darabokra (chunk) bontani a szöveget a beágyazáshoz.")
        try:
            dim = model.get_sentence_embedding_dimension()
            return torch.zeros(dim).numpy()
        except:
             return None

    embeddings = model.encode(chunks, show_progress_bar=False)
    average_embedding = sum(torch.tensor(e) for e in embeddings) / len(embeddings)
    return average_embedding.numpy()


--- 3. Szöveg Tisztítása és Előfeldolgozása ---


In [15]:
# --- Tisztítási lépések alkalmazása ---
df_article_processed['cleaned_text'] = df_article_processed['raw_extracted_text'].apply(clean_special_chars_and_normalize)
df_article_processed['normalized_text'] = df_article_processed['cleaned_text'].str.lower()

# Lemmatizálás
if nlp_hu:
    print("Lemmatizálás folyamatban...")
    df_article_processed['lemmatized_text'] = lemmatize_texts_batch(df_article_processed['normalized_text'].tolist(), nlp_hu)
    df_article_processed['final_processed_text'] = df_article_processed['lemmatized_text']
    print("Lemmatizálás befejezve.")
else:
    print("Figyelem: A spaCy modell nem érhető el, a lemmatizálás kimarad. A 'normalized_text' lesz a 'final_processed_text'.")
    df_article_processed['lemmatized_text'] = df_article_processed['normalized_text']
    df_article_processed['final_processed_text'] = df_article_processed['normalized_text']

print("Tisztított szöveg eleje:")
print(df_article_processed['cleaned_text'].iloc[0][:500])
print("\nFeldolgozott (lemmatizált/normalizált) szöveg eleje:")
print(df_article_processed['final_processed_text'].iloc[0][:500])

Lemmatizálás folyamatban...
Lemmatizálás befejezve.
Tisztított szöveg eleje:
By Natalie Galan In the heart of Alberta, the bison, both plains and wood bison, are more than just iconic symbols of the Canadian West.

Feldolgozott (lemmatizált/normalizált) szöveg eleje:
by natalie galan in the heart of alberta the bis both plains and wood bis are more than just iconic symbols of the canadian west


## 3.5. Szövegkategorizálás (Téma szerint)

In [16]:
print("\n--- 3.5. Szövegkategorizálás (Téma szerint) ---")

def categorize_text_zero_shot(text_to_categorize, candidate_labels, model_name="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", truncation=True):
    classifier = None
    selected_model = model_name
    print(f"Kategorizáló modell ({selected_model}) betöltése...")
    try:
        classifier = pipeline("zero-shot-classification", model=selected_model, device=0 if torch.cuda.is_available() else -1)
        print(f"Modell ({selected_model}) sikeresen betöltve.")
    except Exception as e:
        print(f"Hiba történt a zero-shot klasszifikációs modell ({selected_model}) betöltése közben: {e}")
        alternative_model = "joeddav/xlm-roberta-large-xnli"
        if selected_model != alternative_model:
            print(f"Próbálkozás egy alternatív modellel: {alternative_model}")
            try:
                classifier = pipeline("zero-shot-classification", model=alternative_model, device=0 if torch.cuda.is_available() else -1)
                selected_model = alternative_model
                print(f"Alternatív modell ({selected_model}) sikeresen betöltve.")
            except Exception as e2:
                print(f"Hiba történt az alternatív zero-shot klasszifikációs modell ({selected_model}) betöltése közben is: {e2}")
                return "Kategorizálás sikertelen (modellhiba)", 0.0, selected_model
        else:
             return "Kategorizálás sikertelen (modellhiba)", 0.0, selected_model

    if not classifier:
        return "Kategorizálás sikertelen (nincs modell)", 0.0, selected_model

    print(f"Szöveg kategorizálása a következő címkékkel: {candidate_labels}")
    try:
        max_chars_for_classification = 3000 # Kb. első 500-600 szó
        if len(text_to_categorize) > max_chars_for_classification:
            text_input_for_classifier = text_to_categorize[:max_chars_for_classification]
        else:
            text_input_for_classifier = text_to_categorize

        result = classifier(text_input_for_classifier, candidate_labels, multi_label=False, truncation=truncation)
        predicted_label = result['labels'][0]
        confidence_score = result['scores'][0]
        # print(f"Prediktált kategória ({selected_model} modellel): {predicted_label} (konfidencia: {confidence_score:.4f})")
        return predicted_label, confidence_score, selected_model
    except Exception as e:
        print(f"Hiba történt a zero-shot klasszifikáció végrehajtása során ({selected_model}): {e}")
        return "Kategorizálás sikertelen (futtatási hiba)", 0.0, selected_model

# Kategorizálandó szöveg (a 'cleaned_text' használata ajánlott)
text_for_categorization = df_article_processed['cleaned_text'].iloc[0]

# Témakörök / Kategóriák magyarul
candidate_topics_hu = [
    "Űrkutatás", "Tudomány", "Technológia", "Egészségügy", "Életmód",
    "Biológia", "Környezetvédelem", "Élelmiszertudomány", "Innováció",
    "Kutatás", "Gazdaság", "Politika", "Kultúra", "Sport"
]

if text_for_categorization and text_for_categorization.strip():
    predicted_topic, topic_confidence, used_model = categorize_text_zero_shot(
        text_for_categorization,
        candidate_topics_hu
    )
    df_article_processed['predicted_topic'] = predicted_topic
    df_article_processed['topic_confidence'] = topic_confidence
    df_article_processed['categorization_model'] = used_model
    print(f"A cikk valószínűsített témája ({used_model} modell alapján): {predicted_topic} (Konfidencia: {topic_confidence:.2f})")
else:
    print("Nincs szöveg a kategorizáláshoz.")
    df_article_processed['predicted_topic'] = "N/A"
    df_article_processed['topic_confidence'] = 0.0
    df_article_processed['categorization_model'] = "N/A"


--- 3.5. Szövegkategorizálás (Téma szerint) ---
Kategorizáló modell (MoritzLaurer/mDeBERTa-v3-base-mnli-xnli) betöltése...


Device set to use cpu


Modell (MoritzLaurer/mDeBERTa-v3-base-mnli-xnli) sikeresen betöltve.
Szöveg kategorizálása a következő címkékkel: ['Űrkutatás', 'Tudomány', 'Technológia', 'Egészségügy', 'Életmód', 'Biológia', 'Környezetvédelem', 'Élelmiszertudomány', 'Innováció', 'Kutatás', 'Gazdaság', 'Politika', 'Kultúra', 'Sport']
A cikk valószínűsített témája (MoritzLaurer/mDeBERTa-v3-base-mnli-xnli modell alapján): Kultúra (Konfidencia: 0.30)


## 4. Vektor Reprezentáció (Beágyazások) Generálása

In [17]:
print("\n--- 4. Vektor Reprezentáció Generálása ---")

# Beágyazó modell betöltése
sbert_model = None
sbert_model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
try:
    print(f"Beágyazó modell betöltése: {sbert_model_name}")
    sbert_model = SentenceTransformer(sbert_model_name)
    print(f"Beágyazó modell ({sbert_model_name}) sikeresen betöltve.")
except Exception as e:
    print(f"Hiba a '{sbert_model_name}' beágyazó modell betöltése közben: {e}")

# A 'final_processed_text' (lemmatizált vagy csak normalizált) használata a beágyazáshoz.
text_for_embedding = df_article_processed['final_processed_text'].iloc[0]
article_embedding_vector = None

if sbert_model and text_for_embedding and text_for_embedding.strip():
    print(f"Beágyazás generálása a feldolgozott szöveghez ('final_processed_text')...")
    article_embedding_vector = get_average_embedding(text_for_embedding, sbert_model)
    if article_embedding_vector is not None:
        print("Generált beágyazás alakja:", article_embedding_vector.shape)
        # print("Beágyazás első 5 értéke:", article_embedding_vector[:5])
        df_article_processed['embedding_vector'] = [article_embedding_vector]
    else:
        print("Nem sikerült beágyazást generálni.")
        df_article_processed['embedding_vector'] = [None]
else:
    if not sbert_model:
        print("A beágyazó modell nem töltődött be, a beágyazás generálása kimarad.")
    else:
        print("Nincs feldolgozott szöveg a beágyazáshoz.")
    df_article_processed['embedding_vector'] = [None]


--- 4. Vektor Reprezentáció Generálása ---
Beágyazó modell betöltése: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Beágyazó modell (sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) sikeresen betöltve.
Beágyazás generálása a feldolgozott szöveghez ('final_processed_text')...
Generált beágyazás alakja: (384,)


## 5. Eredmények Mentése és Megjelenítése

In [18]:
print("\n--- Feldolgozás vége ---")

# DataFrame megjelenítése (hasznos lehet notebookban)
print("\nVégső DataFrame releváns oszlopokkal:")
pd.set_option('display.max_colwidth', 80)
try:
    from IPython.display import display
    display(df_article_processed[['id', 'predicted_topic', 'topic_confidence', 'categorization_model', 'final_processed_text']])
except ImportError:
    print(df_article_processed[['id', 'predicted_topic', 'topic_confidence', 'categorization_model', 'final_processed_text']].head())


output_filename_processed = "feldolgozott_szoveg.txt"
with open(output_filename_processed, "w", encoding="utf-8") as f:
    f.write(df_article_processed['final_processed_text'].iloc[0])
print(f"\nA feldolgozott (lemmatizált/normalizált) szöveg elmentve ide: {output_filename_processed}")

output_filename_raw = "nyers_szoveg.txt"
with open(output_filename_raw, "w", encoding="utf-8") as f:
    f.write(df_article_processed['raw_extracted_text'].iloc[0])
print(f"A nyers, kinyert szöveg elmentve ide: {output_filename_raw}")

output_filename_cleaned = "tisztitott_szoveg.txt"
with open(output_filename_cleaned, "w", encoding="utf-8") as f:
    f.write(df_article_processed['cleaned_text'].iloc[0])
print(f"A tisztított (de nem lemmatizált) szöveg elmentve ide: {output_filename_cleaned}")

print(f"\nKategorizálás eredménye: Téma = {df_article_processed['predicted_topic'].iloc[0]}, Konfidencia = {df_article_processed['topic_confidence'].iloc[0]:.2f}, Modell = {df_article_processed['categorization_model'].iloc[0]}")
if df_article_processed['embedding_vector'].iloc[0] is not None:
    print(f"Beágyazásvektor generálva, alakja: {df_article_processed['embedding_vector'].iloc[0].shape}")
else:
    print("Beágyazásvektor nem került generálásra.")


--- Feldolgozás vége ---

Végső DataFrame releváns oszlopokkal:


,id,predicted_topic,topic_confidence,categorization_model,final_processed_text
0,https://www.aiwc.ca/blog/duckweed-a-superfood-from-the-wetlands/,Kultúra,0.300325,MoritzLaurer/mDeBERTa-v3-base-mnli-xnli,by natalie galan in the heart of alberta the bis both plains and wood bis ar...



A feldolgozott (lemmatizált/normalizált) szöveg elmentve ide: feldolgozott_szoveg.txt
A nyers, kinyert szöveg elmentve ide: nyers_szoveg.txt
A tisztított (de nem lemmatizált) szöveg elmentve ide: tisztitott_szoveg.txt

Kategorizálás eredménye: Téma = Kultúra, Konfidencia = 0.30, Modell = MoritzLaurer/mDeBERTa-v3-base-mnli-xnli
Beágyazásvektor generálva, alakja: (384,)
